# Writing out a USGSCSM ISD from a PDS3 LRO LROC NAC image

In [2]:
import ale
from ale.drivers.lro_drivers import LroLrocPds3LabelNaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
import os

ImportError: cannot import name 'config' from 'ale' (/home/kdlee/builds/ale/ale/__init__.py)

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The default config file is located at `ale/config.yml` and is copied into your home directory at `.ale/config.yml` on first use of the library. The config file can be modified using a text editor. `ale.config` is loaded into memory as a dictionary. It is used to find metakernels for different missions. For example, there is an entry for LRO that points to `/usgs/cpkgs/isis3/data/lro/kernels/mk/` by default. If you want to use your own metakernels, you will need to udpate this path. For example, if the metakernels are located in `/data/lrolrocnac/mk/` the LRO entry should be updated with this path. If you are using the default metakernels, then you do not need to update the path.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `LroLrocPds3LabelNaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 LRO LROC NAC image
 * NAIF metakernels installed
 * Config file path for LRO (ale.config.lro) pointing to LRO NAIF metakernel directory 
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [2]:
# printing config displays the yaml formatted string
print(ale.config)

# config object is a dictionary so it has the same access patterns 
print('LRO spice directory:', ale.config['lro'])

# updating config for new LRO path in this notebook 
# Note: this will not change the path in `.ale/config.yml`. This change only lives in the notebook.
# ale.config['lro'] = '/data/lrolrocnac/mk/'

cassini: /usgs/cpkgs/isis3/data/cassini/kernels/mk/
dawn: /data/spice/dawn-m_a-spice-6-v1.0/dawnsp_1000/extras/mk
kaguya: /data/spice/SELENE/kernels/mk/
lro: /scratch/jlaura/spice/lro-l-spice-6-v1.0/lrosp_1000/extras/mk/
mdis: /data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk
mro: /data/spice/mro-m-spice-6-v1.0/mrosp_1000/extras/mk
spice_root: /data/spice/

LRO spice directory: /scratch/jlaura/spice/lro-l-spice-6-v1.0/lrosp_1000/extras/mk/


In [3]:
# change to desired PDS3 image path 
fileName = 'M1142142198RE.IMG'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with LroLrocPds3LabelNaifSpiceDriver(fileName) as driver:
    # pass driver instance into formatter function
    usgscsmString = to_usgscsm(driver)

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.


In [4]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)